# Crear SHP de EM_CONAGUA #

In [11]:
import os
import pandas as pd
from tqdm import tqdm

# Función para extraer datos climatológicos desde un archivo TXT
def extraer_datos_climatologicos(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
    
    # Encontrar el inicio de los datos climatológicos
    inicio_datos = None
    for i, line in enumerate(lines):
        if line.strip().startswith('Fecha'):
            inicio_datos = i + 1
            break

    if inicio_datos is None:
        return pd.DataFrame()

    # Extraer y procesar los datos climatológicos
    data = []
    for line in lines[inicio_datos:]:
        parts = line.split()
        if len(parts) >= 5:
            try:
                fecha = parts[0]
                dia, mes, año = fecha.split('/')
                dia = int(dia)
                mes = int(mes)
                año = int(año)
                if año >= 2000 and año <= 2020:  # Filtrar por el rango de fechas
                    precip = parts[1]
                    tmax = parts[3] if parts[3] != 'Nulo' else None
                    tmin = parts[4] if parts[4] != 'Nulo' else None
                    data.append([dia, mes, año, precip, tmin, tmax])
            except ValueError:
                continue  # Ignorar líneas con formato de fecha incorrecto

    # Convertir a DataFrame
    df = pd.DataFrame(data, columns=['Dia', 'Mes', 'Año', 'Precip', 'TMin', 'TMax'])
    df['Precip'] = pd.to_numeric(df['Precip'], errors='coerce')
    df['TMin'] = pd.to_numeric(df['TMin'], errors='coerce')
    df['TMax'] = pd.to_numeric(df['TMax'], errors='coerce')
    
    return df

# Función para convertir los archivos TXT a CSV, solo para las estaciones en el rango de fechas deseado
def convertir_txt_a_csv(carpeta_txt, carpeta_csv, metadatos_path):
    # Leer los metadatos
    metadatos = pd.read_csv(metadatos_path)
    metadatos['fecha_inicial'] = pd.to_datetime(metadatos['fecha_inicial'])
    metadatos['fecha_final'] = pd.to_datetime(metadatos['fecha_final'])
    
    # Filtrar estaciones por fecha
    fecha_inicio = pd.Timestamp('2000-01-01')
    fecha_fin = pd.Timestamp('2020-12-31')
    metadatos_filtrados = metadatos[(metadatos['fecha_inicial'] <= fecha_inicio) & (metadatos['fecha_final'] >= fecha_fin)]
    
    # Obtener lista de estaciones filtradas en formato de 5 dígitos
    estaciones_filtradas = metadatos_filtrados['estacion'].astype(str).str.zfill(5).values
    
    # Crear la carpeta de salida si no existe
    os.makedirs(carpeta_csv, exist_ok=True)
    
    # Obtener lista de archivos TXT en la carpeta y extraer los IDs de estación en formato de 5 dígitos
    archivos_txt = [os.path.splitext(f)[0][3:].zfill(5) for f in os.listdir(carpeta_txt) if f.startswith('dia') and f.endswith('.TXT')]
    
    # Barra de progreso
    for archivo in tqdm(os.listdir(carpeta_txt), desc="Procesando archivos"):
        if archivo.startswith('dia') and archivo.endswith('.TXT'):
            file_path = os.path.join(carpeta_txt, archivo)
            
            # Obtener ID de la estación desde el nombre del archivo y ajustar al formato de 5 dígitos
            estacion_id = os.path.splitext(archivo)[0][3:].zfill(5)
            
            # Verificar si la estación está en los metadatos filtrados
            if estacion_id in estaciones_filtradas:
                # Extraer datos climatológicos
                df = extraer_datos_climatologicos(file_path)
                
                # Verificar que el DataFrame contiene datos desde el 1 de enero de 2000 hasta el 31 de diciembre de 2020
                if not df.empty and df['Año'].min() <= 2000 and df['Año'].max() >= 2020:
                    archivo_csv = os.path.splitext(archivo)[0] + '.csv'
                    df.to_csv(os.path.join(carpeta_csv, archivo_csv), index=False)

# Parámetros de entrada
carpeta_txt = '/Users/DiegoRB/Desktop/Proyectos/EM_Mex/txts' 
carpeta_csv = '/Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021'  
metadatos_path = '/Users/DiegoRB/Desktop/Proyectos/EM_Mex/Metadatos/metadatos_final_2.csv' 

# Convertir los archivos
convertir_txt_a_csv(carpeta_txt, carpeta_csv, metadatos_path)

Procesando archivos: 100%|██████████████████| 5445/5445 [02:08<00:00, 42.38it/s]


In [12]:
# Ruta de la carpeta
carpeta = '/Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021'

# Contar archivos en la carpeta
cantidad_archivos = len([f for f in os.listdir(carpeta) if os.path.isfile(os.path.join(carpeta, f))])

print(f"La cantidad de archivos en la carpeta es: {cantidad_archivos}")


La cantidad de archivos en la carpeta es: 1058


In [15]:
import os
import pandas as pd


metadatos = pd.read_csv(metadatos_path)

# Convertir las fechas de operación a formato datetime
metadatos['fecha_inicial'] = pd.to_datetime(metadatos['fecha_inicial'])
metadatos['fecha_final'] = pd.to_datetime(metadatos['fecha_final'])

# Definir el rango de fechas
fecha_inicio = pd.Timestamp('2000-01-01')
fecha_fin = pd.Timestamp('2020-12-31')

# Filtrar estaciones que cumplen con el criterio
metadatos_filtrados = metadatos[(metadatos['fecha_inicial'] <= fecha_inicio) & (metadatos['fecha_final'] >= fecha_fin)]
estaciones_filtradas = metadatos_filtrados['estacion'].astype(str).str.zfill(5).values

# Obtener lista de archivos CSV generados
archivos_csv = [os.path.splitext(f)[0][3:].zfill(5) for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]

# Verificar qué estaciones tienen archivos CSV generados y cuáles faltan
estaciones_con_archivo = [estacion for estacion in estaciones_filtradas if estacion in archivos_csv]
estaciones_sin_archivo = [estacion for estacion in estaciones_filtradas if estacion not in archivos_csv]

# Resultados
print(f"Cantidad de estaciones con archivos CSV generados: {len(estaciones_con_archivo)}")
print(f"Cantidad de estaciones sin archivos CSV: {len(estaciones_sin_archivo)}")
print("Estaciones sin archivos CSV:", estaciones_sin_archivo[:160])  # Mostrar las primeras 10 estaciones que faltan


Cantidad de estaciones con archivos CSV generados: 1058
Cantidad de estaciones sin archivos CSV: 143
Estaciones sin archivos CSV: ['01077', '01089', '02031', '02034', '02055', '02070', '02138', '02152', '04002', '04011', '04025', '04030', '04036', '04052', '05157', '05180', '06007', '06062', '08182', '09010', '09045', '10143', '12077', '12106', '12113', '12125', '13008', '13015', '13024', '13091', '13128', '13130', '13144', '14075', '14099', '14139', '14149', '14154', '14189', '14368', '15028', '15037', '15046', '15047', '15050', '15059', '15074', '15077', '15081', '15090', '15100', '15101', '15103', '15106', '15125', '15130', '15133', '15135', '15137', '15167', '15263', '15274', '15283', '16003', '16026', '16027', '16078', '16174', '16238', '16512', '17022', '17062', '17063', '17064', '17065', '17066', '18007', '18010', '18012', '18015', '18044', '18072', '18076', '19004', '19016', '19036', '19053', '19068', '19136', '19142', '19152', '19166', '19172', '20003', '20030', '20073', '2009

In [16]:
estaciones_sin_archivo

['01077',
 '01089',
 '02031',
 '02034',
 '02055',
 '02070',
 '02138',
 '02152',
 '04002',
 '04011',
 '04025',
 '04030',
 '04036',
 '04052',
 '05157',
 '05180',
 '06007',
 '06062',
 '08182',
 '09010',
 '09045',
 '10143',
 '12077',
 '12106',
 '12113',
 '12125',
 '13008',
 '13015',
 '13024',
 '13091',
 '13128',
 '13130',
 '13144',
 '14075',
 '14099',
 '14139',
 '14149',
 '14154',
 '14189',
 '14368',
 '15028',
 '15037',
 '15046',
 '15047',
 '15050',
 '15059',
 '15074',
 '15077',
 '15081',
 '15090',
 '15100',
 '15101',
 '15103',
 '15106',
 '15125',
 '15130',
 '15133',
 '15135',
 '15137',
 '15167',
 '15263',
 '15274',
 '15283',
 '16003',
 '16026',
 '16027',
 '16078',
 '16174',
 '16238',
 '16512',
 '17022',
 '17062',
 '17063',
 '17064',
 '17065',
 '17066',
 '18007',
 '18010',
 '18012',
 '18015',
 '18044',
 '18072',
 '18076',
 '19004',
 '19016',
 '19036',
 '19053',
 '19068',
 '19136',
 '19142',
 '19152',
 '19166',
 '19172',
 '20003',
 '20030',
 '20073',
 '20096',
 '20129',
 '20177',
 '20271',


In [19]:
import os
import pandas as pd
from tqdm import tqdm

# Función para extraer datos climatológicos desde un archivo TXT
def extraer_datos_climatologicos(file_path):
    with open(file_path, 'r', encoding='ISO-8859-1') as file:
        lines = file.readlines()
    
    # Encontrar el inicio de los datos climatológicos
    inicio_datos = None
    for i, line in enumerate(lines):
        if line.strip().startswith('Fecha'):
            inicio_datos = i + 1
            break

    if inicio_datos is None:
        return pd.DataFrame()

    # Extraer y procesar los datos climatológicos
    data = []
    for line in lines[inicio_datos:]:
        parts = line.split()
        if len(parts) >= 5:
            try:
                fecha = parts[0]
                dia, mes, año = fecha.split('/')
                dia = int(dia)
                mes = int(mes)
                año = int(año)
                if año >= 2000 and año <= 2020:  # Filtrar por el rango de fechas
                    precip = parts[1]
                    tmax = parts[3] if parts[3] != 'Nulo' else None
                    tmin = parts[4] if parts[4] != 'Nulo' else None
                    data.append([dia, mes, año, precip, tmin, tmax])
            except ValueError:
                continue  # Ignorar líneas con formato de fecha incorrecto

    # Convertir a DataFrame
    df = pd.DataFrame(data, columns=['Dia', 'Mes', 'Año', 'Precip', 'TMin', 'TMax'])
    df['Precip'] = pd.to_numeric(df['Precip'], errors='coerce')
    df['TMin'] = pd.to_numeric(df['TMin'], errors='coerce')
    df['TMax'] = pd.to_numeric(df['TMax'], errors='coerce')
    
    return df

# Función para verificar las estaciones faltantes
def verificar_estaciones_omitidas(carpeta_txt, estaciones_sin_archivo):
    problemas_estaciones = {}
    for estacion_id in tqdm(estaciones_sin_archivo, desc="Verificando estaciones faltantes"):
        file_path = os.path.join(carpeta_txt, f'dia{estacion_id}.TXT')
        if os.path.exists(file_path):
            df = extraer_datos_climatologicos(file_path)
            if df.empty:
                problemas_estaciones[estacion_id] = "Datos vacíos o fuera del rango de fechas"
            elif df['Año'].min() > 2000 or df['Año'].max() < 2020:
                problemas_estaciones[estacion_id] = "Datos no cubren todo el rango de fechas"
            else:
                problemas_estaciones[estacion_id] = "Otro problema desconocido"
        else:
            problemas_estaciones[estacion_id] = "Archivo TXT no encontrado"
    
    return problemas_estaciones


# Leer los metadatos y obtener estaciones filtradas
metadatos = pd.read_csv(metadatos_path)
metadatos['fecha_inicial'] = pd.to_datetime(metadatos['fecha_inicial'])
metadatos['fecha_final'] = pd.to_datetime(metadatos['fecha_final'])
fecha_inicio = pd.Timestamp('2000-01-01')
fecha_fin = pd.Timestamp('2020-12-31')
metadatos_filtrados = metadatos[(metadatos['fecha_inicial'] <= fecha_inicio) & (metadatos['fecha_final'] >= fecha_fin)]
estaciones_filtradas = metadatos_filtrados['estacion'].astype(str).str.zfill(5).values

# Obtener lista de archivos CSV generados
archivos_csv = [os.path.splitext(f)[0][3:].zfill(5) for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]

# Verificar qué estaciones tienen archivos CSV generados y cuáles faltan
estaciones_sin_archivo = [estacion for estacion in estaciones_filtradas if estacion not in archivos_csv]

# Verificar las estaciones faltantes
problemas_estaciones = verificar_estaciones_omitidas(carpeta_txt, estaciones_sin_archivo)

# Mostrar algunos resultados
print(f"Cantidad de estaciones omitidas: {len(problemas_estaciones)}")
for estacion, problema in list(problemas_estaciones.items())[:10]:
    print(f"Estación: {estacion}, Problema: {problema}")


Verificando estaciones faltantes: 100%|███████| 143/143 [00:05<00:00, 24.91it/s]

Cantidad de estaciones omitidas: 143
Estación: 01077, Problema: Datos no cubren todo el rango de fechas
Estación: 01089, Problema: Datos no cubren todo el rango de fechas
Estación: 02031, Problema: Datos no cubren todo el rango de fechas
Estación: 02034, Problema: Datos no cubren todo el rango de fechas
Estación: 02055, Problema: Datos no cubren todo el rango de fechas
Estación: 02070, Problema: Datos no cubren todo el rango de fechas
Estación: 02138, Problema: Datos no cubren todo el rango de fechas
Estación: 02152, Problema: Datos no cubren todo el rango de fechas
Estación: 04002, Problema: Datos no cubren todo el rango de fechas
Estación: 04011, Problema: Datos no cubren todo el rango de fechas


In [28]:
import os
import pandas as pd

# Función para revisar la estructura de datos
def revisar_estructura(file_path):
    try:
        df = pd.read_csv(file_path)
        print(f"Primeras filas de {file_path}:")
        print(df.head())
        print(f"Columnas presentes: {df.columns}")
    except Exception as e:
        print(f"Error al leer {file_path}: {e}")

# Revisar la estructura de algunos archivos CSV
archivos_csv = [f for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]
for archivo in archivos_csv[:5]:  # Revisar solo los primeros 5 archivos
    file_path = os.path.join(carpeta_csv, archivo)
    revisar_estructura(file_path)


Primeras filas de /Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021/dia08142.csv:
   Dia  Mes   Año  Precip  TMin  TMax
0    1    1  2000     0.0  -3.0  12.6
1    2    1  2000     0.0  -6.0  14.4
2    3    1  2000     0.0  -9.0  17.6
3    4    1  2000     0.0 -11.0  15.8
4    5    1  2000     0.0 -11.4  18.0
Columnas presentes: Index(['Dia', 'Mes', 'Año', 'Precip', 'TMin', 'TMax'], dtype='object')
Primeras filas de /Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021/dia08156.csv:
   Dia  Mes   Año  Precip  TMin  TMax
0    1    1  2000     0.0   6.0  26.0
1    2    1  2000     0.0  -1.0  27.0
2    3    1  2000     0.0  -1.0  27.0
3    4    1  2000     0.0  -5.0  20.0
4    5    1  2000     0.0  -4.0  13.0
Columnas presentes: Index(['Dia', 'Mes', 'Año', 'Precip', 'TMin', 'TMax'], dtype='object')
Primeras filas de /Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021/dia19067.csv:
   Dia  Mes   Año  Precip  TMin  TMax
0    1    1  2000     0.0   3.0  23.0
1    2    1  2000     0.0

In [34]:
import os
import pandas as pd
from tqdm import tqdm

# Función para agregar columna de fecha y guardar el CSV usando nombres de columna
def agregar_columna_fecha(file_path):
    try:
        df = pd.read_csv(file_path)
        if all(col in df.columns for col in ['Dia', 'Mes', 'Año']):
            # Verificar valores de las columnas antes de convertir
            print(f"Procesando {file_path}")
            print("Valores únicos en 'Dia':", df['Dia'].unique())
            print("Valores únicos en 'Mes':", df['Mes'].unique())
            print("Valores únicos en 'Año':", df['Año'].unique())
            
            # Convertir columnas a numéricas en caso de que haya errores de formato
            df['Dia'] = pd.to_numeric(df['Dia'], errors='coerce')
            df['Mes'] = pd.to_numeric(df['Mes'], errors='coerce')
            df['Año'] = pd.to_numeric(df['Año'], errors='coerce')
            
            # Verificar nuevamente después de la conversión
            print("Después de conversión:")
            print("Valores únicos en 'Dia':", df['Dia'].unique())
            print("Valores únicos en 'Mes':", df['Mes'].unique())
            print("Valores únicos en 'Año':", df['Año'].unique())
            
            # Crear la columna de fecha
            df['Fecha'] = pd.to_datetime(df[['Año', 'Mes', 'Dia']])
            
            # Reordenar las columnas para tener 'Fecha' al inicio
            df = df[['Fecha'] + [col for col in df.columns if col != 'Fecha']]
            df.to_csv(file_path, index=False)
        else:
            print(f"Columnas faltantes en {file_path}. Columnas presentes: {df.columns}")
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        # Mostrar algunas filas del archivo problemático para depuración
        try:
            df_sample = pd.read_csv(file_path).head()
            print(df_sample)
        except Exception as e2:
            print(f"Error al leer el archivo {file_path} para depuración: {e2}")

# Parámetros de entrada
carpeta_csv = '/Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021'   # Ajusta esta ruta según tu entorno

# Agregar columna de fecha a los archivos CSV
archivos_csv = [f for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]
for archivo in archivos_csv[:1]:  # Revisar solo el primer archivo para diagnóstico
    file_path = os.path.join(carpeta_csv, archivo)
    agregar_columna_fecha(file_path)


Procesando /Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021/dia08142.csv
Valores únicos en 'Dia': [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
Valores únicos en 'Mes': [ 1  2  3  4  5  6  7  8  9 10 11 12]
Valores únicos en 'Año': [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020]
Después de conversión:
Valores únicos en 'Dia': [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]
Valores únicos en 'Mes': [ 1  2  3  4  5  6  7  8  9 10 11 12]
Valores únicos en 'Año': [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020]
Error procesando /Users/DiegoRB/Desktop/Proyectos/EM_Mex/csvs_2000_2021/dia08142.csv: to assemble mappings requires at least that [year, month, day] be specified: [day,month,year] is missing
   Dia  Mes   Año  Precip  TMin  TMax
0    1    1  2000     0.0  -3.0  12

In [35]:
import os
import pandas as pd
from tqdm import tqdm

# Función para agregar columna de fecha y guardar el CSV
def agregar_columna_fecha(file_path):
    try:
        df = pd.read_csv(file_path)
        if all(col in df.columns for col in ['Dia', 'Mes', 'Año']):
            # Crear la columna de fecha concatenando las columnas Dia, Mes y Año
            df['Fecha'] = pd.to_datetime(df['Año'].astype(str) + '-' + df['Mes'].astype(str) + '-' + df['Dia'].astype(str), errors='coerce')
            # Reordenar las columnas para tener 'Fecha' al inicio
            df = df[['Fecha'] + [col for col in df.columns if col != 'Fecha']]
            df.to_csv(file_path, index=False)
        else:
            print(f"Columnas faltantes en {file_path}. Columnas presentes: {df.columns}")
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        # Mostrar algunas filas del archivo problemático para depuración
        try:
            df_sample = pd.read_csv(file_path).head()
            print(df_sample)
        except Exception as e2:
            print(f"Error al leer el archivo {file_path} para depuración: {e2}")

# Agregar columna de fecha a los archivos CSV
archivos_csv = [f for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]
for archivo in tqdm(archivos_csv, desc="Agregando columna de fecha a CSVs"):
    file_path = os.path.join(carpeta_csv, archivo)
    agregar_columna_fecha(file_path)


Agregando columna de fecha a CSVs: 100%|████| 1058/1058 [01:44<00:00, 10.10it/s]


In [36]:
import os
import pandas as pd
from tqdm import tqdm

# Función para verificar presencia completa de fechas en un archivo CSV
def verificar_fechas_completas(file_path, fecha_inicio, fecha_fin):
    try:
        df = pd.read_csv(file_path)
        if 'Fecha' in df.columns:
            df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
        else:
            print(f"Columna 'Fecha' faltante en {file_path}")
            return False, []
        
        # Crear un rango de fechas completo
        fechas_completas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')
        
        # Verificar si todas las fechas están presentes en el DataFrame
        fechas_df = df['Fecha'].dropna().unique()
        fechas_faltantes = fechas_completas.difference(fechas_df)
        
        return len(fechas_faltantes) == 0, fechas_faltantes
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return False, []

fecha_inicio = pd.Timestamp('2000-01-01')
fecha_fin = pd.Timestamp('2020-12-31')

# Verificar presencia completa de fechas en los archivos CSV
archivos_csv = [f for f in os.listdir(carpeta_csv) if f.startswith('dia') and f.endswith('.csv')]

resultados_verificacion = {}
for archivo in tqdm(archivos_csv, desc="Verificando fechas completas en CSVs"):
    file_path = os.path.join(carpeta_csv, archivo)
    fechas_completas, fechas_faltantes = verificar_fechas_completas(file_path, fecha_inicio, fecha_fin)
    resultados_verificacion[archivo] = (fechas_completas, fechas_faltantes)

# Mostrar resultados
archivos_incompletos = [archivo for archivo, (fechas_completas, _) in resultados_verificacion.items() if not fechas_completas]
print(f"Cantidad de archivos CSV con fechas incompletas: {len(archivos_incompletos)}")
print("Archivos CSV con fechas incompletas:", archivos_incompletos[:10])  # Mostrar los primeros 10 archivos con fechas incompletas

# Mostrar las fechas faltantes para los primeros archivos con fechas incompletas
for archivo in archivos_incompletos[:10]:
    _, fechas_faltantes = resultados_verificacion[archivo]
    print(f"Fechas faltantes en {archivo}: {fechas_faltantes[:10]}")  # Mostrar las primeras 10 fechas faltantes


Verificando fechas completas en CSVs: 100%|█| 1058/1058 [00:17<00:00, 59.90it/s]

Cantidad de archivos CSV con fechas incompletas: 996
Archivos CSV con fechas incompletas: ['dia08142.csv', 'dia08156.csv', 'dia19067.csv', 'dia19098.csv', 'dia22026.csv', 'dia20143.csv', 'dia15305.csv', 'dia30185.csv', 'dia14033.csv', 'dia02137.csv']
Fechas faltantes en dia08142.csv: DatetimeIndex(['2002-01-01', '2002-01-02', '2002-01-03', '2002-01-04',
               '2002-01-05', '2002-01-06', '2002-01-07', '2002-01-08',
               '2002-01-09', '2002-01-10'],
              dtype='datetime64[ns]', freq=None)
Fechas faltantes en dia08156.csv: DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10'],
              dtype='datetime64[ns]', freq=None)
Fechas faltantes en dia19067.csv: DatetimeIndex(['2003-10-31', '2003-11-01', '2003-11-02', '2003-11-03',
               '2003-11-04', '2003-11-05', '2003-11-06', '2003-11-07',
               '2003-11-08', '2003-